In [9]:
import os
import gradio as gr
import numpy as np
import pandas as pd
import cv2
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from PIL import Image

# 📁 Load CSV Data
df = pd.read_csv("/content/synthetic_genetic_disorder_data.csv")

# 🧠 Encode labels
label_encoder = LabelEncoder()
df["Label"] = label_encoder.fit_transform(df["Disease"])
disease_mapping = {i: name for i, name in enumerate(label_encoder.classes_)}

# 🖼️ Load and preprocess all images
image_data = []
labels = []

for idx, row in df.iterrows():
    img_path = os.path.join("/content", row["ImageFile"])  # Change if image path is different
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    if img is not None:
        img = cv2.resize(img, (128, 128))
        img = img / 255.0
        img = img.reshape(128, 128, 1)
        image_data.append(img)
        labels.append(row["Label"])
    else:
        print(f"Failed to load image: {img_path}")

# ✅ Convert to arrays
image_data = np.array(image_data)
labels = to_categorical(np.array(labels))

# 🧠 CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    MaxPooling2D(),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(labels.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 🏋️ Train model
X_train, X_test, y_train, y_test = train_test_split(image_data, labels, test_size=0.2, random_state=42)
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# 🔍 Predict Function
def predict_from_image_only(uploaded_img):
    img = uploaded_img.convert("L").resize((128, 128))
    img_array = np.array(img) / 255.0
    img_array = img_array.reshape(1, 128, 128, 1)

    prediction = model.predict(img_array)[0]
    predicted_class = int(np.argmax(prediction))
    disease_name = disease_mapping.get(predicted_class, "Unknown")

    return f"Predicted Disease: {disease_name}"

# 🧪 Gradio Interface
iface = gr.Interface(
    fn=predict_from_image_only,
    inputs=gr.Image(type="pil", label="Upload an Image"),
    outputs=gr.Textbox(label="Prediction Result"),
    title="Image-Only Genetic Disease Predictor",
    description="Upload a medical image to predict the genetic disorder."
)

iface.launch()


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 1.8344 - val_accuracy: 0.0000e+00 - val_loss: 3.9361
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step - accuracy: 0.2500 - loss: 2.1997 - val_accuracy: 0.0000e+00 - val_loss: 4.0106
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 335ms/step - accuracy: 0.2500 - loss: 1.3141 - val_accuracy: 0.0000e+00 - val_loss: 4.1144
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 256ms/step - accuracy: 0.5000 - loss: 1.2620 - val_accuracy: 0.0000e+00 - val_loss: 3.5140
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 308ms/step - accuracy: 1.0000 - loss: 0.5499 - val_accuracy: 0.0000e+00 - val_loss: 3.5318
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step - accuracy: 1.0000 - loss: 0.4681 - val_accuracy: 0.0000e+00 - val_loss: 3.5924
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 311ms/step - accuracy: 0.5000 - loss: 0.9562 - val_accuracy: 0.0000e+00 - val_loss: 3.4860
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step - accuracy: 0.7500 - loss: 0.5992 - val_accuracy: 